# ARGOS Multi-Unit Hotel Demo (Colab-ready)

This notebook demonstrates the multi-property version of ARGOS,
using the `MultiUnitHotelSystem` wrapper.


In [ ]:
import os
os.chdir('/content/argos_repo') if os.path.exists('/content/argos_repo') else None


In [ ]:
!pip install -q -r requirements.txt


In [ ]:
import sys, os
if 'src' not in os.listdir('.'):
    print("Warning: run this from the repo root (where `src/` lives).")
sys.path.append(os.path.abspath('src'))


In [ ]:
from argos.hotel_env import HotelEnvironment
from argos.lco import LexicographicTier, LexicographicTierStructure
from argos.cag import CAGFilter
from argos.argos_core import ARGOSOptimizer
from argos.multiunit import MultiUnitHotelSystem
import numpy as np


In [ ]:
def revenue_grad(x: np.ndarray) -> np.ndarray:
    g = np.zeros_like(x)
    g[0] = - (50 + 100 * x[3])
    g[3] = - (100 * x[0])
    return g

def newton_step(x: np.ndarray) -> np.ndarray:
    H_inv = np.array([1e-3, 1.0, 1.0, 1e-3])
    return -H_inv * revenue_grad(x)

def contour_step(x: np.ndarray) -> np.ndarray:
    g = revenue_grad(x)
    norm = np.linalg.norm(g) + 1e-8
    return -0.1 * g / norm

def make_argos():
    tier1 = LexicographicTier(
        name="Feasibility",
        objective=lambda x: 0.0,
        constraints=[lambda x: max(0.0, x[0] - 1.0)],
    )
    tiers = LexicographicTierStructure([tier1])
    cag = CAGFilter(newton_step_fn=newton_step, contour_step_fn=contour_step)
    return ARGOSOptimizer(tiers=tiers, cag=cag, step_size=0.05, highest_tier=1)


In [ ]:
envs = [HotelEnvironment(seed=100 + i) for i in range(3)]
optims = [make_argos() for _ in envs]
system = MultiUnitHotelSystem(envs=envs, optimizers=optims)

states = system.reset()
history = []

for t in range(100):
    grads = [revenue_grad for _ in states]
    out = system.step(states, grads)
    states = out["states"]
    for pid, m in enumerate(out["metrics"]):
        m = dict(m)
        m["epoch"] = t
        m["property_id"] = pid
        history.append(m)

len(history)


In [ ]:
import pandas as pd
df = pd.DataFrame(history)
df.head()


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 5))
for pid, grp in df.groupby('property_id'):
    ax.plot(grp['epoch'], grp['revpar'], label=f'Property {pid}')
ax.set_title('RevPAR by Property')
ax.legend()
plt.show()
